In [1]:
import wandb
from sumolib import checkBinary
import Genetic_Algorithm as GA
import numpy as np
import matplotlib.pyplot as plt

(b'<configuration><input><net-file value="Manhattan5x3.net.xml" /><route-files '
 b'value="Manhattan5x3.rou.xml" /></input><time><begin value="0" /><end value="'
 b'1000" /></time><time-to-teleport value="-1" /></configuration>')
Launching SUMO


In [2]:
#Initialize Weights and Biases
wandb.init(project="IA", entity="davidredquin")

#SUMO parameters
sumo_binary = checkBinary('sumo')
sumo_binary_gui = checkBinary('sumo-gui')
config_file = 'tutorial.sumocfg'

pop_size = 12
max_generations = 20  
n_survivors = 3

#Mutation and Crossover parameters
crossover_rate=0.5
duration_mutation_rate=0.2
duration_mutation_strength=15
states_mutation_rate = 0.2
light_options = ['G', 'y', 'r']


#Fitness Function parameters
collision_penalty   = [400,4000,10000,400,4000]
waiting_time_weight = [10/20000,15/2000,5/2000,20/2000]
emissions_weight    = [10/20000,5/2000,15/2000,0/2000]


# Launch 5 simulated experiments
total_runs = 1
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
    wandb.init(
      # Set the project where this run will be logged
      project="IA", 
      entity="davidredquin",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"GA_22_{run}",
      # Track hyperparameters and run metadata
      config={
        "config_file":config_file,
        "collision_penalty": collision_penalty[run],
        "waiting_time_weight": waiting_time_weight[run],
        "emissions_weight": emissions_weight[run],
        "architecture": "Genetic Algorithm",
        "population_size": pop_size,
        "max_generations":max_generations,
        "n_survivors":n_survivors,
        "crossover_rate":crossover_rate,
        "duration_mutation_rate":duration_mutation_rate,
        "duration_mutation_strength":duration_mutation_strength,
        "states_mutation_rate":states_mutation_rate       
      }
    )

    #Get Base chromosome
    chromosome_base = GA.get_chromosome()
    GA_pop_next = GA.generate_random_population(chromosome_base)
    current_gen = 0
    #Generations=[]
    
    #Run GA
    while(current_gen <max_generations):
        GA_pop = GA_pop_next
        
        
        ## Evaluating Population
        fitness, emissions, waiting = GA_pop.evaluate_pop()
        #Generations.append([GA_pop.generation,GA_pop.best_fitness,GA_pop.best_emissions,GA_pop.best_waiting_time,
        #                    GA_pop.avg_fitness,GA_pop.avg_emissions,GA_pop.avg_waiting_time])

        print("Best Fitness value in Generation ",current_gen, " is ",GA_pop.best_fitness)

        #Logging data to Weights and Biases
        best_fitness = GA_pop.best_fitness
        best_emissions = GA_pop.best_emissions
        best_waiting_time = GA_pop.best_waiting_time
        avg_fitness = GA_pop.avg_fitness
        avg_emissions = GA_pop.avg_emissions
        avg_waiting_time = GA_pop.avg_waiting_time

        print("Logging info to WandB")
        wandb.log({"best_fitness":best_fitness,"best_emissions":best_emissions,"best_waiting_time":best_waiting_time,
                   "avg_fitness":avg_fitness, "avg_emissions":avg_emissions,"avg_waiting_time":avg_waiting_time})
        
        
        ## Visualize SUMO run
        if current_gen // 5 == 0:
            GA.visualize_SUMO(GA_pop.best_individual)
        
        #Calculating Survivors
        sorted_pop = np.argsort(fitness)[:n_survivors]
        Next_gene_pool = [GA_pop.gene_pool[idx] for idx in sorted_pop]
        
        #Creating new Generation
        while len(Next_gene_pool) < pop_size:

            chromosome_male =GA_pop.gene_pool[int(np.random.uniform(0,n_survivors))]
            chromosome_female=GA_pop.gene_pool[int(np.random.uniform(0,n_survivors))]
            chromosome_child = GA.crossover_parent(chromosome_male, chromosome_female,crossover_rate)
            chromosome_child = GA.mutate_chromosome(chromosome_child,0.1,5,0.2)
            Next_gene_pool.append(chromosome_child)

        current_gen = current_gen+1
        GA_pop_next = GA.population(current_gen)
        GA_pop_next.gene_pool = Next_gene_pool

    wandb.finish()

KeyboardInterrupt: 

In [4]:
vehiclewaiting_media,result_vehiclewaiting=GA.visualize_SUMO()
vehiclewaiting_media2,result_vehiclewaiting2=GA.visualize_SUMO(GA_pop.best_individual)
result_vehiclewaiting = np.nanmean(vehiclewaiting_media)
result_vehiclewaiting2 = np.nanmean(vehiclewaiting_media2)

# Crear una gráfica
plt.figure()
# Añadir una línea horizontal roja discontinua en la posición de result_personwaiting
plt.axhline(y=result_vehiclewaiting, color='b', linestyle='--')
plt.plot(vehiclewaiting_media)
plt.legend([f'Tiempo de espera modo normal: {result_vehiclewaiting}',f'Tiempo de espera AG: {result_vehiclewaiting2}'])

# Añadir una línea horizontal roja discontinua en la posición de result_personwaiting
plt.axhline(y=result_vehiclewaiting2, color='g', linestyle='--')
plt.plot(vehiclewaiting_media2,color='g')
plt.show()